In [1]:
import pandas as pd
import sys
import os.path
import numpy as np
import cProfile
import pstats
import matplotlib.pyplot as plt
import bokeh.plotting
from bokeh.plotting import figure, output_notebook, show, gridplot, reset_output

import spykshrk.realtime.binary_record as bin_rec

pd.set_option('float_format', '{:,.1f}'.format)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

output_notebook()

In [ ]:
rec_dir = '/opt/data36/daliu/realtime/spykshrk/bond04'
rec_dir = '/opt/data36/daliu/realtime/spykshrk/timing'
# rec_dir = '/home/daliu/output/test'
mpi_ranks = [0, 2, 3, 4, 5, 6]
file_prefix = 'test_animal'
manager_label = 'state'
file_postfix = 'bin_rec'

allrecs = {}

for mpi_rank in mpi_ranks:
    binrec = bin_rec.BinaryRecordsFileReader(save_dir=rec_dir, file_prefix=file_prefix,
                                             mpi_rank=mpi_rank,
                                             manager_label=manager_label,
                                             file_postfix=file_postfix, filemeta_as_col=False)

    rec_pandas = binrec.convert_pandas()
    
    for rec_id, rec_pd in rec_pandas.items():
        existing_rec_pd = allrecs.setdefault(rec_id, pd.DataFrame())
        allrecs[rec_id] = existing_rec_pd.append(rec_pd, ignore_index=True)
    
for rec_id, rec_pd in allrecs.items():
    if 'timestamp' in rec_pd.columns:
        rec_pd.sort_values(['timestamp']).reset_index(drop=True)

In [16]:
allrecs.keys()

In [17]:
spike_dec = allrecs[4]
spike_dec

In [19]:
p1 = plt.plot(spike_dec[spike_dec['timestamp'] == 102141000].loc[:, 'x0':'x149'].values[0])

plt.show(p1)

In [72]:
decode_values = spike_dec.values[:,2:].astype('float')
decode_values = decode_values + 1e-200
decode_values = decode_values * 1/np.max(decode_values, axis=1)[:,None]

In [76]:

plt.figure(figsize=[20,20])
im1 = plt.imshow(np.transpose(decode_values), aspect='auto')
plt.show(im1)